In [76]:
import os
import csv
import mysql.connector as mc 
import time
import webbrowser
import sys
from sklearn.cluster import KMeans
%matplotlib inline
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
sns.set()
from functools import reduce

In [85]:
cnx = mc.connect(user='root',password='***',host='localhost',database='mlb')

this took total time of  3.999962091445923


In [102]:
tester = REMatrix(cnx,yr=2017,mo=5)

In [106]:
tester2.expect_dict

{0: 0.53900000000000003,
 1: 0.91800000000000004,
 2: 1.161,
 3: 1.46,
 4: 1.4490000000000001,
 5: 1.6359999999999999,
 6: 1.79,
 7: 2.113,
 8: 0.28399999999999997,
 9: 0.53600000000000003,
 10: 0.67100000000000004,
 11: 0.90000000000000002,
 12: 1.0760000000000001,
 13: 1.0649999999999999,
 14: 1.4430000000000001,
 15: 1.5489999999999999,
 16: 0.13,
 17: 0.254,
 18: 0.36499999999999999,
 19: 0.50900000000000001,
 20: 0.38200000000000001,
 21: 0.503,
 22: 0.55000000000000004,
 23: 0.76400000000000001}

In [105]:
startTime= time.time()
tester2 = REMatrix(cnx,yr=2017,mo=6)
print ('this took {} seconds'.format(time.time()-startTime))

this took 95.31803965568542 seconds


In [101]:
class REMatrix:
    #df is the full dataframe while game_inn_df just captures the 
    #distinct game and innings df. all_combos is that in list form 
    def __init__(self,cnx,yr=None,mo=None):#,game_inn_df,df):
        query1=''
        query2=''
        if yr and mo:
            query1 = """SELECT Pitch2.gameID AS gameID ,curr_inn FROM Pitch2 INNER JOIN Game
                        ON Pitch2.gameID = Game.gameID
                        WHERE YEAR(gameDate)='{}' AND MONTH(gameDate)='{}'
                        GROUP BY gameID,curr_inn""".format(yr,mo)
            
            query2 = """SELECT Game.gameID AS gameID,abNum,pitchID,the_event,firstBaseRunner,
            secondBaseRunner, thirdBaseRunner,outs,home_team_runs,
            away_team_runs,curr_inn, homeTeamScore,awayTeamScore,gameDate
            FROM Pitch2 INNER JOIN Game ON Pitch2.gameID = Game.gameID
            WHERE YEAR(gameDate)='{}' AND MONTH(gameDate)='{}'""".format(yr,mo)
        elif yr:
            query1 = """SELECT Pitch2.gameID AS gameID ,curr_inn FROM Pitch2 INNER JOIN Game
                        ON Pitch2.gameID = Game.gameID WHERE YEAR(gameDate)='{}'
                        GROUP BY gameID,curr_inn""".format(yr)
            
            query2 = """SELECT Game.gameID AS gameID,abNum,pitchID,the_event,firstBaseRunner,
            secondBaseRunner, thirdBaseRunner,outs,home_team_runs, away_team_runs,curr_inn, 
            homeTeamScore,awayTeamScore,gameDate FROM Pitch2 INNER JOIN Game 
            ON Pitch2.gameID = Game.gameID WHERE YEAR(gameDate)='{}'""".format(yr,mo)
        else:
            query1 = """SELECT Pitch2.gameID AS gameID ,curr_inn FROM Pitch2"""
            
            query2 = """SELECT Game.gameID AS gameID,abNum,pitchID,the_event,firstBaseRunner,
            secondBaseRunner, thirdBaseRunner,outs,home_team_runs,
            away_team_runs,curr_inn, homeTeamScore,awayTeamScore,gameDate FROM Pitch2"""
        
        self.game_inn_df = pd.read_sql_query(query1,cnx)
        self.df = pd.read_sql_query(query2,cnx)
        #self.game_inn_df = game_inn_df
        self.all_combos = [(item[1],item[2]) for item 
                           in self.game_inn_df.itertuples()] 
        #self.df = df 
        self.inning_list = []
        self.ab_list = []
        self.expect_dict = {}
        self.generate_ab_list()
        self.calculate_run_expectancies()
        self.add_run_expectancies()
    
    def add_run_expectancies(self):
        #theoretically the at-bats should be in correct order...I would hope at least
        for i,ab in enumerate(self.ab_list):
            start_sit = ab.situation 
            ab.start_run_expectancy = self.expect_dict[start_sit]
            try:
                ab.next_ab = self.ab_list[i+1]
                if ab.next_ab.abNum == ab.abNum+1:
                    ab.end_run_expectancy = ab.next_ab.start_run_expectancy
                else:
                    ab.end_run_expectancy = 0
            except IndexError as e: #special case for the last item
                ab.next_ab = None
                ab.end_run_expectancy = 0
        
    def generate_ab_list(self):
        for j,item in enumerate(self.all_combos):
            #create the dataframe for one half inning of a game
            #and use it to create an inning object which we'll then pass
            #to a list 
            inning_df = self.df[(self.df.loc[:,'gameID']==item[0]) & 
                          (self.df.loc[:,'curr_inn']==item[1])]
            inning_df = inning_df.sort_values(by=['pitchID'],kind='quicksort')
            inning_object = Inning(inning_df)
            self.inning_list.append(inning_object)

            for i in range(len(inning_df)):
                try:
                #if ab number is not the same as the next one, it's last pitch of AB
                    if inning_df.iloc[i,1]<inning_df.iloc[i+1,1]:    
                        try:
                            ab_object = AB(inning_df.iloc[i,:],inning_object)
                            self.ab_list.append(ab_object)
                        #i.e. this is the condition where there's sometimes more than 3 outs...
                        except ValueError as e:
                            continue
                        except Exception as e1:
                            print ('tried to create an AB and got exception: ',e1)

                #last pitch of inning was made, so it must be last pitch of AB
                except IndexError as e:
                    try:
                        ab_object = AB(inning_df.iloc[i,:],inning_object)
                        self.ab_list.append(ab_object)
                    except ValueError as e4:
                        print ('again we hit this at end of inning',e4)
                except Exception as e3:
                    print ('some other error that was not an index error in the inning_df: ',e3)
    
    def calculate_run_expectancies(self):
        situations = [item.situation for item in self.ab_list]
        runs_created = [item.runs_created for item in self.ab_list]
        run_expectancy_df = pd.DataFrame({'situation':situations,'runs_created':runs_created})
        for i in range(24):
            self.expect_dict[i] = round(run_expectancy_df[run_expectancy_df.situation==i].mean()['runs_created'],3)

In [81]:
class Inning:
    def __init__(self,df):
        self.inning_number = df['curr_inn'].mode()[0]
        self.gameID = df['gameID'].mode()[0]
        self.home_or_away = self.get_home_or_away()
        self.endScore = self.get_end_score(df)

    def get_home_or_away(self):
        if self.inning_number*2%2 == 1:
            return 'Home'
        else:
            return 'Away'

    def get_end_score(self,df):
        #just look at the last row of the dataframe
        if self.home_or_away == 'Home':
            return df['home_team_runs'].tail(1).iloc[0]
        else:
            return df['away_team_runs'].tail(1).iloc[0]


In [82]:
class AB:
    #passing in a series here. 
    def __init__(self,df,inning):
        
        self.gameID = df['gameID']
        self.abNum = df['abNum']
        self.outs = df['outs']
        self.runners = (df['firstBaseRunner'],df['secondBaseRunner'],df['thirdBaseRunner'])
        self.event = df['the_event']
        self.inning = inning
        self.situation = self.calculate_state()
        self.runs_created  = self.calculate_runs_created(df)
        self.start_run_expectancy = 0
        self.end_run_expectancy = 0
        self.next_ab = None
        self.starting_runs = None
        self.adjusted_runs_created = None
    
    #only call this after you've created a run-expectancy instance
    def calculate_adjusted_runs_created(self):
        if not next_ab:
            self.adjusted_runs_created = self.end_run_expectancy - self.start_run_expectancy
        else:
            self.adjusted_runs_created = (self.end_run_expectancy -
                                          self.start_run_expectancy + 
                                          self.next_ab.starting_runs -
                                          self.starting_runs)
            
    def calculate_runs_created(self,df):
        if self.inning.home_or_away == 'Home':
            self.starting_runs = df['home_team_runs']
            return self.inning.endScore - df['home_team_runs']
        else:
            self.starting_runs = df['away_team_runs']
            return self.inning.endScore - df['away_team_runs']

    #determines which of the 24 runner on base/outs circumstance
    def calculate_state(self):
        total = 0
        if self.runners[0]:
            total+=1
        if self.runners[1]:
            total+=2
        if self.runners[2]:
            total+=4
        if self.outs>=3 or self.outs<0:
            raise ValueError("cant start AB with less than 0 or > 3 outs")
        if self.outs == 1:
            total+=8
        elif self.outs ==2:
            total+=16
        return total
    

In [73]:
df[(df.loc[:,'thirdBaseRunner'].isnull()==False) &
    (df.loc[:,'secondBaseRunner'].isnull()==False) &
    (df.loc[:,'firstBaseRunner'].isnull()==False)].head()

,gameID,abNum,pitchID,the_event,firstBaseRunner,secondBaseRunner,thirdBaseRunner,outs,home_team_runs,away_team_runs,curr_inn,homeTeamScore,awayTeamScore,gameDate
193,ANA201706010,55,429,Flyout,519184,455117,527043,2,1,1,7.5,2,4,2017-06-01
194,ANA201706010,55,430,Flyout,519184,455117,527043,2,1,1,7.5,2,4,2017-06-01
195,ANA201706010,55,431,Flyout,519184,455117,527043,2,1,1,7.5,2,4,2017-06-01
237,ANA201706010,69,539,Single,500871,592696,593871,1,2,1,9.0,2,4,2017-06-01
238,ANA201706010,69,540,Single,500871,592696,593871,1,2,1,9.0,2,4,2017-06-01


In [107]:
cnx.close()